In [1]:

from utils.utils import get_documents_from_index, convert_text_to_ascii
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
)
from pydantic import BaseModel, Field
from typing import Literal
# from ..globals import *
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from typing import Literal
import os


In [ ]:


# Workflow constants
NUM_FOLLOWUP_Q = 3
TOP_N_DOCUMENTS = 5

# LLM Stuff
API_KEY = os.getenv("OPENAI_KEY")
AZURE_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
API_VERSION = "2025-01-01-preview"


compare_questions_human = """
#USER_QUESTION: {user_query}

#index_query_1: {index_query_1}
#index_query_2: {index_query_2}
#index_query_3: {index_query_3}
#index_query_4: {index_query_4}
#index_query_5: {index_query_5}

"""
prompt = """Your task is to evaluate the semantic similarity between a given user query (which may contain multiple parts or compound questions) and a list of indexed questions. The goal is to determine whether the meaning and intent of the user query and each indexed question are the same or different within the context of the medical domain for UnitedHealth Group.  
  
### **Independent Evaluation:**  
- Assess each indexed question independently.  
- Do not let the content or context of previous indexed questions influence your evaluation of the current question.  
- Treat each indexed question as a standalone comparison with the user query.  
- **Compound/Multiple-Part Queries**:   
  - If the user query contains multiple parts, the match is only **True** if the indexed question aligns with **all parts** of the user query.   
  - If any part of the user query does not align with the indexed question, the match must be **False**.  
  
### **Key Requirements:**  
- **Semantic Similarity in the Medical Domain**:  
  - Evaluate alignment based on meaning and intent, even if the wording differs.  
  - Pay close attention to healthcare-specific differences in terminology, such as "coverage," "cost," "copay," "coinsurance," "provider," and "specialist."  
  - Ensure that the **context** of the medical domain remains consistent across all parts of the user query and indexed question.  
  - For compound queries, the intent of **every part** of the user query must match the indexed question for a **True** result.   
  - If an indexed question addresses only **one part** of a compound query while ignoring other parts, the match must be **False**.  
  
### **Decision-Making Criteria:**  
- **True**: The user query and indexed question express the same meaning and intent across **all parts** of the user query (terminologies match, context matches, and specific focuses align for every part).  
- **False**: The user query and indexed question express different meanings or intents, **or if any part of a compound query does not align** with the indexed question.  
  
### **Examples of Common Scenarios:**  
1. **Exact Match of All Parts**:  
   - User Query: "Is eyewear covered under the plan and does it cover home health care?"  
   - Indexed Question: "Is eyewear covered under the plan and does it cover home health care?"  
   - **True** (Both parts match the intent and meaning exactly).  
  
2. **Partial Match for Compound Questions**:  
   - User Query: "Is eyewear covered under the plan and does it cover home health care?"  
   - Indexed Question 1: "Is eyewear covered under this plan?"  
   - Indexed Question 2: "Does this plan cover home health care?"  
   - **False** (Each indexed question addresses only one part of the compound query and ignores the other part, making it a partial match).  
  
3. **Different Focus**:  
   - User Query: "Is eyewear covered under the plan and does it cover home health care?"  
   - Indexed Question: "Does this plan cover general eye exams?"  
   - **False** (The user query focuses on eyewear coverage, while the indexed question focuses on eye exams, leading to different intents).  
  
4. **Different Terminology in Compound Query**:  
   - User Query: "Can I see a provider and is the copay affordable?"  
   - Indexed Question: "Can I see a provider?"  
   - **False** (The first part aligns, but the second part about copay affordability does not match).  
  
**Handling Ambiguity:**  
- If a question is ambiguous or unclear, use the context of the medical domain and the intent of the user query to make a reasoned decision.  
- If intent cannot be reasonably inferred, default to False and provide a clear explanation.  
- The user query and indexed question have partial match it should be False.
  
**Output Format:**  
For each indexed question, provide a True or False response along with reasoning in JSON format. Ensure that your evaluation for each question is independent.   
  
Example output:  
```json  
{  
    "is_similar1": "False",  
    "reason1": "The user query is about seeing doctors out of network, while the indexed question is unrelated.",  
    "is_similar2": "True",  
    "reason2": "The user query aligns with the coverage question for out of network doctors.",  
    "is_similar3": "False",  
    "reason3": "The user query is about seeing out of network providers and eye cost, which is partial match.",  
    "is_similar4": "False",  
    "reason4": "The user query addresses the ability to see a doctor, while the indexed question is about specific costs.",  
    "is_similar5": "False",  
    "reason5": "The user query is about general doctors, while the indexed question specifies a specialist."  
} ``` """

from typing import Literal
class SimilarityData(BaseModel):  
    is_similar1: Literal[True, False]  
    is_similar2: Literal[True, False]  
    is_similar3: Literal[True, False]  
    is_similar4: Literal[True, False]  
    is_similar5: Literal[True, False]  
    Reason1: str
    Reason2: str
    Reason3: str
    Reason4: str
    Reason5: str

	  
compare_questions_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(prompt),
        HumanMessagePromptTemplate.from_template(
            template=compare_questions_human,
            input_variables=["user_query", "index_query_1", "index_query_2", "index_query_3", "index_query_4", "index_query_5"]
        ),
    ]
)

	  
llm = AzureChatOpenAI(
    api_key=API_KEY,  # type:ignore
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="gpt-4o-mini",
    api_version=API_VERSION,
    temperature=0,
)

compare_questions_chain = compare_questions_template | llm.with_structured_output(SimilarityData, method="json_schema")
async def async_compare_questions(user_query: str, index_query_1: str, index_query_2: str, index_query_3: str, index_query_4: str, index_query_5: str):
    print("user_query:" + user_query)
    print("index_query_1: " + index_query_1)
    print("index_query_2: " + index_query_2)
    print("index_query_3: " + index_query_3)
    print("index_query_4: " + index_query_4)
    print("index_query_5: " + index_query_5)
    """
    Asynchronously compares two questions using the compare_questions_chain.

    Args:
        user_query (str): The user's query to be compared.
        index_query (str): The index query to compare against.

    Returns:
        tuple: A tuple containing the user query, index query, and the response from the comparison.
    """

    response = await compare_questions_chain.ainvoke(
        input={"user_query": user_query,
                "index_query_1": index_query_1,
                "index_query_2": index_query_2, 
                "index_query_3": index_query_3, 
               "index_query_4": index_query_4, 
               "index_query_5": index_query_5}
    )
    response
    return response
  
	  



In [3]:

user_query = "What are the copayments or coinsurance amounts for my regular doctor visits and hospital stays?"
index_query_1 = "How much will it cost to see my doctor?"
index_query_2 = "How much will it cost if I go to the hospital?"
index_query_3 = "Does this plan offer OTC?"
index_query_4 = "Does this plan cover crutches?"
index_query_5 = "What is my out of pocket maximum?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

user_query:What are the copayments or coinsurance amounts for my regular doctor visits and hospital stays?
index_query_1: How much will it cost to see my doctor?
index_query_2: How much will it cost if I go to the hospital?
index_query_3: Does this plan offer OTC?
index_query_4: Does this plan cover crutches?
index_query_5: What is my out of pocket maximum?
('is_similar1', False)
('is_similar2', False)
('is_similar3', False)
('is_similar4', False)
('is_similar5', False)
('Reason1', 'The user query asks specifically about copayments or coinsurance for regular doctor visits, while the indexed question only asks about the cost to see a doctor, which does not address the specific details of copayments or coinsurance.')
('Reason2', 'The user query inquires about copayments or coinsurance for hospital stays, but the indexed question only asks about the cost of going to the hospital, lacking the specific focus on copayments or coinsurance.')
('Reason3', 'The user query is focused on copayment

In [4]:
user_query = "is eyewear covered under the plan and does it cover home health care? "
index_query_1 =  "Is eyewear covered under this plan?"
index_query_2 = "Does this plan cover home health care?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

user_query:is eyewear covered under the plan and does it cover home health care? 
index_query_1: Is eyewear covered under this plan?
index_query_2: Does this plan cover home health care?
index_query_3: Does this plan offer OTC?
index_query_4: Does this plan cover crutches?
index_query_5: What is my out of pocket maximum?
('is_similar1', False)
('is_similar2', False)
('is_similar3', False)
('is_similar4', False)
('is_similar5', False)
('Reason1', 'The user query asks about eyewear coverage and home health care, while the indexed question only addresses eyewear coverage, making it a partial match.')
('Reason2', 'The user query includes a question about home health care coverage, while the indexed question only addresses eyewear coverage, resulting in a partial match.')
('Reason3', 'The user query focuses on eyewear and home health care, while the indexed question is about OTC offerings, which is unrelated.')
('Reason4', 'The user query is about eyewear and home health care coverage, whil

In [5]:
index_query_1 = "What's the cost to see a provider?"

index_query_2 = "What would it cost me to see a doc?"

# index_query_3 = "Provider costs?"
index_query_3 = "What's the copay to see a doctor?"

index_query_5 = "How much would it cost to see a specialist?"

index_query_4 = "What's the coinsurance for PCP visits?"
user_query = "How much does it cost to see a doctor?"

In [6]:
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:How much does it cost to see a doctor?
index_query_1: What's the cost to see a provider?
index_query_2: What would it cost me to see a doc?
index_query_3: What's the copay to see a doctor?
index_query_4: What's the coinsurance for PCP visits?
index_query_5: How much would it cost to see a specialist?


SimilarityData(is_similar1=True, is_similar2=True, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query and indexed question both inquire about the cost of seeing a provider, aligning in meaning and intent.', Reason2='The user query and indexed question both ask about the cost associated with seeing a doctor, maintaining the same intent.', Reason3='The user query is about the overall cost to see a doctor, while the indexed question specifically asks about the copay, which is a different aspect of cost.', Reason4='The user query focuses on the general cost to see a doctor, whereas the indexed question is specifically about coinsurance for PCP visits, which does not match the broader inquiry.', Reason5='The user query is about the cost to see a doctor in general, while the indexed question specifically asks about the cost to see a specialist, which is a different focus.')

In [7]:
index_query_3 = "Provider costs?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:How much does it cost to see a doctor?
index_query_1: What's the cost to see a provider?
index_query_2: What would it cost me to see a doc?
index_query_3: Provider costs?
index_query_4: What's the coinsurance for PCP visits?
index_query_5: How much would it cost to see a specialist?


SimilarityData(is_similar1=True, is_similar2=True, is_similar3=True, is_similar4=False, is_similar5=False, Reason1='The user query and indexed question both inquire about the cost of seeing a provider, aligning in meaning and intent.', Reason2='The user query and indexed question both ask about the cost to see a doctor, maintaining the same intent and context.', Reason3='The user query and indexed question both focus on the costs associated with seeing a provider, thus they are semantically similar.', Reason4='The user query is about the general cost to see a doctor, while the indexed question specifically asks about coinsurance for PCP visits, which is a different focus.', Reason5='The user query is about the cost to see a doctor in general, while the indexed question specifically asks about the cost to see a specialist, which does not align with the intent of the user query.')

In [8]:
user_query = "Does this plan offer OTC?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan offer OTC?
index_query_1: What's the cost to see a provider?
index_query_2: What would it cost me to see a doc?
index_query_3: Provider costs?
index_query_4: What's the coinsurance for PCP visits?
index_query_5: How much would it cost to see a specialist?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query is about over-the-counter (OTC) offerings, while the indexed question is focused on costs to see a provider, which is unrelated.', Reason2='The user query inquires about OTC benefits, whereas the indexed question is about the cost of seeing a doctor, leading to different intents.', Reason3='The user query is about OTC options, while the indexed question addresses provider costs, which do not align in meaning or intent.', Reason4='The user query focuses on OTC offerings, while the indexed question is about coinsurance for primary care provider visits, making them unrelated.', Reason5='The user query is about OTC benefits, while the indexed question pertains to costs associated with seeing a specialist, resulting in different focuses.')

In [9]:
user_query = "Is eyewear covered under this plan?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Is eyewear covered under this plan?
index_query_1: What's the cost to see a provider?
index_query_2: What would it cost me to see a doc?
index_query_3: Provider costs?
index_query_4: What's the coinsurance for PCP visits?
index_query_5: How much would it cost to see a specialist?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query is about eyewear coverage, while the indexed question is about the cost to see a provider, which is unrelated.', Reason2='The user query focuses on eyewear coverage, whereas the indexed question inquires about the cost to see a doctor, leading to different intents.', Reason3="The user query is specifically about eyewear coverage, while the indexed question addresses provider costs, which does not align with the user's intent.", Reason4='The user query is about coverage for eyewear, while the indexed question pertains to coinsurance for primary care provider visits, making it unrelated.', Reason5="The user query asks about eyewear coverage, while the indexed question is about the cost to see a specialist, which does not match the user's intent.")

In [10]:
index_query_1 =  "Does this plan offer OTC?"

index_query_2 = "Does this plan cover home health care?"

index_query_3 = "Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?"

index_query_4 = "Can I see doctors out of the network?"
index_query_5 = "Is eyewear covered under this plan?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Is eyewear covered under this plan?
index_query_1: Does this plan offer OTC?
index_query_2: Does this plan cover home health care?
index_query_3: Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?
index_query_4: Can I see doctors out of the network?
index_query_5: Is eyewear covered under this plan?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=True, Reason1='The user query is specifically about eyewear coverage, while the indexed question is about OTC, which is unrelated.', Reason2='The user query focuses on eyewear coverage, whereas the indexed question addresses home health care, leading to different intents.', Reason3='The user query is about eyewear coverage, while the indexed question pertains to benefits for chronic conditions, which does not align.', Reason4='The user query is about eyewear coverage, while the indexed question is about seeing out-of-network doctors, making it unrelated.', Reason5='The indexed question directly matches the user query regarding eyewear coverage under the plan.')

In [11]:

user_query = "Can I see providers out of the network?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Can I see providers out of the network?
index_query_1: Does this plan offer OTC?
index_query_2: Does this plan cover home health care?
index_query_3: Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?
index_query_4: Can I see doctors out of the network?
index_query_5: Is eyewear covered under this plan?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=True, is_similar5=False, Reason1='The user query is about seeing providers out of network, while the indexed question is unrelated to that topic.', Reason2='The user query is about seeing providers out of network, while the indexed question focuses on home health care coverage, which is a different subject.', Reason3="The user query is about seeing providers out of network, while the indexed question addresses special benefits for chronic conditions, which does not align with the user's intent.", Reason4="The indexed question directly addresses the user's query about seeing doctors out of the network, matching the intent and meaning.", Reason5='The user query is about seeing providers out of network, while the indexed question is about eyewear coverage, which is unrelated.')

In [12]:
user_query = "Does this plan offer OTC?"
index_query_4 = "Can I see doctors out of the network?"
index_query_5 = "Can I see an OON specialist?"
index_query_3 = "Are OON docs covered??"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan offer OTC?
index_query_1: Does this plan offer OTC?
index_query_2: Does this plan cover home health care?
index_query_3: Are OON docs covered??
index_query_4: Can I see doctors out of the network?
index_query_5: Can I see an OON specialist?


SimilarityData(is_similar1=True, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The indexed question matches the user query exactly, both asking if the plan offers OTC (over-the-counter) benefits.', Reason2='The indexed question is about home health care coverage, which is unrelated to the user query about OTC benefits.', Reason3='The indexed question addresses coverage for out-of-network doctors, which does not relate to the user query about OTC.', Reason4='The indexed question is about seeing doctors out of the network, which is not relevant to the user query regarding OTC.', Reason5='The indexed question focuses on seeing an out-of-network specialist, which does not align with the user query about OTC.')

In [13]:
user_query = "Does this plan offer OTC?"
index_query_1 = "Can I see providers outside of the network?"
index_query_2 = "Are OON docs covered?"
index_query_3 = "Out of network providers?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
# index_query_5 = "What's the coinsurance for out of network PCP visits?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan offer OTC?
index_query_1: Can I see providers outside of the network?
index_query_2: Are OON docs covered?
index_query_3: Out of network providers?
index_query_4: What's the copay for out of network doctors?
index_query_5: Can I see an OON specialist?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query is about OTC (over-the-counter) offerings, while the indexed question is about seeing providers outside of the network, which is unrelated.', Reason2='The user query focuses on OTC offerings, whereas the indexed question is about coverage for out-of-network doctors, leading to different intents.', Reason3="The user query is about OTC, while the indexed question is a general inquiry about out-of-network providers, which does not align with the user's intent.", Reason4='The user query addresses OTC offerings, while the indexed question is specifically about copay for out-of-network doctors, making it unrelated.', Reason5="The user query is about OTC, while the indexed question pertains to seeing an out-of-network specialist, which does not match the user's intent.")

In [14]:



user_query = "Can I see doctors out of the network?"
index_query_1 = "Does this plan  offer OTC"
index_query_2 = "Are OON docs covered?"
index_query_3 = "Out of network providers?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

user_query:Can I see doctors out of the network?
index_query_1: Does this plan  offer OTC
index_query_2: Are OON docs covered?
index_query_3: Out of network providers?
index_query_4: What's the copay for out of network doctors?
index_query_5: Can I see an OON specialist?
('is_similar1', False)
('is_similar2', True)
('is_similar3', True)
('is_similar4', False)
('is_similar5', True)
('Reason1', 'The user query is about seeing doctors out of the network, while the indexed question is unrelated to that topic.')
('Reason2', "The indexed question directly addresses whether out of network doctors are covered, aligning with the user's inquiry about seeing them.")
('Reason3', "The indexed question refers to out of network providers, which matches the user's question about seeing doctors out of the network.")
('Reason4', 'The user query does not inquire about copay amounts, making this indexed question only partially relevant.')
('Reason5', "The indexed question about seeing an out of network sp

In [15]:
user_query = "What are the copayments or coinsurance amounts for my regular doctor visits and hospital stays?"
index_query_1 = "How much will it cost to see my doctor?"
index_query_2 = "How much will it cost if I go to the hospital?"
index_query_3 = "Does this plan offer OTC?"
index_query_4 = "Does this plan cover crutches?"
index_query_5 = "What is my out of pocket maximum?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

user_query:What are the copayments or coinsurance amounts for my regular doctor visits and hospital stays?
index_query_1: How much will it cost to see my doctor?
index_query_2: How much will it cost if I go to the hospital?
index_query_3: Does this plan offer OTC?
index_query_4: Does this plan cover crutches?
index_query_5: What is my out of pocket maximum?
('is_similar1', False)
('is_similar2', False)
('is_similar3', False)
('is_similar4', False)
('is_similar5', False)
('Reason1', 'The user query asks specifically about copayments or coinsurance for regular doctor visits, while the indexed question only asks about the cost to see a doctor, which does not address the specific types of payments mentioned.')
('Reason2', 'The user query inquires about copayments or coinsurance for hospital stays, but the indexed question only addresses the cost of going to the hospital without specifying payment types.')
('Reason3', 'The user query is focused on copayments and coinsurance for doctor visit

In [16]:

user_query = "How much will it cost to see my doctor and what is co-insurance for a hospital visit?"
index_query_1 = "How much will it cost to see my doctor?"
index_query_2 = "How much will it cost if I go to the hospital?"
index_query_3 = "Does this plan offer OTC?"
index_query_4 = "Does this plan cover crutches?"
index_query_5 = "What is my out of pocket maximum?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

user_query:How much will it cost to see my doctor and what is co-insurance for a hospital visit?
index_query_1: How much will it cost to see my doctor?
index_query_2: How much will it cost if I go to the hospital?
index_query_3: Does this plan offer OTC?
index_query_4: Does this plan cover crutches?
index_query_5: What is my out of pocket maximum?
('is_similar1', True)
('is_similar2', False)
('is_similar3', False)
('is_similar4', False)
('is_similar5', False)
('Reason1', 'The indexed question directly matches the first part of the user query regarding the cost to see the doctor.')
('Reason2', "The indexed question addresses the cost of going to the hospital, which does not align with the user's inquiry about co-insurance for a hospital visit.")
('Reason3', "The indexed question is unrelated to the user's query about costs and co-insurance.")
('Reason4', "The indexed question is unrelated to the user's query about costs and co-insurance.")
('Reason5', "The indexed question is unrelated 

In [17]:
user_query = "How much does it cost to see a doctor?"
index_query_1 = "Can I see providers outside of the network?"
index_query_2 = "Are OON docs covered?"
index_query_3 = "Out of network providers?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
# index_query_5 = "What's the coinsurance for out of network PCP visits?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:How much does it cost to see a doctor?
index_query_1: Can I see providers outside of the network?
index_query_2: Are OON docs covered?
index_query_3: Out of network providers?
index_query_4: What's the copay for out of network doctors?
index_query_5: Can I see an OON specialist?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query is about the cost of seeing a doctor, while the indexed question is about seeing providers outside of the network, which is a different focus.', Reason2='The user query asks about the cost to see a doctor, while the indexed question is about coverage for out of network doctors, which does not address cost directly.', Reason3='The user query is focused on the cost of seeing a doctor, while the indexed question is simply asking about out of network providers without any mention of cost.', Reason4='The user query is about the cost to see a doctor, while the indexed question specifically asks about the copay for out of network doctors, which is a different aspect of cost.', Reason5='The user query is about the cost to see a doctor in general, while the indexed question is specifically about seeing an out of network specialist, which does not address the over

In [18]:

user_query = "Is eyewear covered under this plan?"
index_query_1 = "Does this plan include coverage for eyewear?"
index_query_2 = "Are glasses or contact lenses covered?"
index_query_3 = "Is vision care included in this plan?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Is eyewear covered under this plan?
index_query_1: Does this plan include coverage for eyewear?
index_query_2: Are glasses or contact lenses covered?
index_query_3: Is vision care included in this plan?
index_query_4: What's the copay for out of network doctors?
index_query_5: Can I see an OON specialist?


SimilarityData(is_similar1=True, is_similar2=True, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query and indexed question both inquire about coverage for eyewear, expressing the same intent and meaning.', Reason2='The user query and indexed question both address the coverage of glasses or contact lenses, aligning perfectly in intent.', Reason3='The user query specifically asks about eyewear coverage, while the indexed question is broader, asking about vision care, which may not necessarily include eyewear.', Reason4='The user query is focused on eyewear coverage, while the indexed question is about copay for out of network doctors, which is unrelated.', Reason5='The user query is about eyewear coverage, while the indexed question is about seeing an out of network specialist, which does not align.')

In [19]:
user_query = "Does this plan offer OTC?"
index_query_1 = "What OTC items are covered under this plan?" #diff
index_query_2 = "How much is the allowance for OTC products?" #diff
index_query_3 = "Can I get OTC medications with this plan?" #diff
index_query_4 = "Does this plan include OTC benefits?"
index_query_5 = "Are over-the-counter items covered?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan offer OTC?
index_query_1: What OTC items are covered under this plan?
index_query_2: How much is the allowance for OTC products?
index_query_3: Can I get OTC medications with this plan?
index_query_4: Does this plan include OTC benefits?
index_query_5: Are over-the-counter items covered?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=True, is_similar5=True, Reason1='The user query asks if the plan offers OTC, while the indexed question specifies which OTC items are covered, making it a different focus.', Reason2='The user query is about whether the plan offers OTC, while the indexed question inquires about the allowance for OTC products, which is a different aspect.', Reason3="The user query is about the availability of OTC medications, while the indexed question asks if they can be obtained, which does not align with the user's intent.", Reason4='The user query aligns with the indexed question as both inquire about the inclusion of OTC benefits in the plan.', Reason5='The user query about whether the plan offers OTC aligns with the indexed question regarding coverage of over-the-counter items.')

In [20]:
user_query = "Does this plan cover home health care?"
index_query_1 = "Is home health care included in this plan?"
index_query_2 = "Does this plan provide home health services?"
index_query_3 = "Are home health care services covered?"
index_query_4 = "What types of home health care are covered?" #diff
index_query_5 = "Is there a limit on home health care visits?" #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan cover home health care?
index_query_1: Is home health care included in this plan?
index_query_2: Does this plan provide home health services?
index_query_3: Are home health care services covered?
index_query_4: What types of home health care are covered?
index_query_5: Is there a limit on home health care visits?


SimilarityData(is_similar1=True, is_similar2=True, is_similar3=True, is_similar4=False, is_similar5=False, Reason1='The user query and indexed question both inquire about the coverage of home health care, expressing the same intent and meaning.', Reason2='The user query and indexed question both ask about the provision of home health services, aligning in intent and meaning.', Reason3='The user query and indexed question both focus on the coverage of home health care services, matching in intent and meaning.', Reason4="The user query asks about coverage, while the indexed question specifically asks about types of home health care, which does not align with the user's intent regarding coverage.", Reason5='The user query is about coverage of home health care, while the indexed question inquires about limits on visits, which is a different focus.')

In [21]:
user_query = "Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?"
index_query_1 = "Are there special programs for chronic conditions in this plan?"
index_query_2 = "What chronic conditions are covered under special programs?" #diff
index_query_3 = "Are there any wellness programs for chronic disease management?" #diff
index_query_4 = "Does this plan include benefits for managing diabetes or heart disease?"
index_query_5 = "Are chronic condition management programs offered?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?
index_query_1: Are there special programs for chronic conditions in this plan?
index_query_2: What chronic conditions are covered under special programs?
index_query_3: Are there any wellness programs for chronic disease management?
index_query_4: Does this plan include benefits for managing diabetes or heart disease?
index_query_5: Are chronic condition management programs offered?


SimilarityData(is_similar1=True, is_similar2=False, is_similar3=False, is_similar4=True, is_similar5=True, Reason1='The user query and indexed question both inquire about special programs for managing chronic conditions, aligning perfectly in intent and meaning.', Reason2='The user query asks about special benefits or programs for managing chronic conditions, while the indexed question specifically asks which chronic conditions are covered, making it a different focus.', Reason3="The user query is about special benefits or programs for chronic conditions, while the indexed question focuses on wellness programs, which may not directly align with the user's intent.", Reason4='The user query mentions managing chronic conditions like diabetes or heart disease, and the indexed question directly addresses benefits for managing those specific conditions, aligning well.', Reason5="The user query asks about special benefits or programs for managing chronic conditions, and the indexed question i

In [22]:
user_query = "Is eyewear covered under this plan and what is copay to visit doctor?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5) 
response

user_query:Is eyewear covered under this plan and what is copay to visit doctor?
index_query_1: Are there special programs for chronic conditions in this plan?
index_query_2: What chronic conditions are covered under special programs?
index_query_3: Are there any wellness programs for chronic disease management?
index_query_4: Does this plan include benefits for managing diabetes or heart disease?
index_query_5: Are chronic condition management programs offered?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The user query is about eyewear coverage and copay for doctor visits, which is unrelated to chronic conditions or programs.', Reason2='The user query focuses on eyewear and copay, while this indexed question is about chronic conditions, leading to different intents.', Reason3='The user query does not address wellness programs or chronic disease management, making it unrelated to the indexed question.', Reason4='The user query is about eyewear and doctor copay, while this indexed question focuses on specific chronic diseases, resulting in a mismatch.', Reason5='The user query is centered on eyewear and doctor visits, which does not align with the topic of chronic condition management programs.')

In [23]:
user_query = "Does this plan cover home health care?"
index_query_1, index_query_2, index_query_3 = 'Is home health care included in this plan?', 'Does this plan provide home health services?', 'Are home health care services covered?'
index_query_4, index_query_5 = 'What types of home health care are covered?', 'Is there a limit on home health care visits?' #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:Does this plan cover home health care?
index_query_1: Is home health care included in this plan?
index_query_2: Does this plan provide home health services?
index_query_3: Are home health care services covered?
index_query_4: What types of home health care are covered?
index_query_5: Is there a limit on home health care visits?


SimilarityData(is_similar1=True, is_similar2=True, is_similar3=True, is_similar4=False, is_similar5=False, Reason1='The user query and indexed question both inquire about the coverage of home health care, expressing the same intent and meaning.', Reason2='The user query and indexed question both ask about the provision of home health services, aligning in intent and meaning.', Reason3='The user query and indexed question both focus on the coverage of home health care services, matching in intent and meaning.', Reason4="The user query asks about coverage, while the indexed question specifically asks about types of home health care, which does not align with the user's intent regarding coverage.", Reason5='The user query is about coverage of home health care, while the indexed question inquires about limits on visits, which is a different focus.')

In [24]:
user_query = "What is my monthly premium and does it cover hearing aid?"
index_query_1 = "What is my monthly premium?"
index_query_2 = " does it cover hearing aid?"
index_query_3 = "What is my monthly premium and does it cover hearing aid?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:What is my monthly premium and does it cover hearing aid?
index_query_1: What is my monthly premium?
index_query_2:  does it cover hearing aid?
index_query_3: What is my monthly premium and does it cover hearing aid?
index_query_4: What types of home health care are covered?
index_query_5: Is there a limit on home health care visits?


SimilarityData(is_similar1=False, is_similar2=False, is_similar3=True, is_similar4=False, is_similar5=False, Reason1='The indexed question only addresses the premium and does not mention hearing aids, making it a partial match.', Reason2='The indexed question only addresses hearing aids and does not mention the premium, making it a partial match.', Reason3='The indexed question matches exactly with both parts of the user query regarding the monthly premium and hearing aid coverage.', Reason4='The indexed question is about home health care coverage, which is unrelated to the user query about premiums and hearing aids.', Reason5='The indexed question is about limits on home health care visits, which does not relate to the user query about premiums and hearing aids.')

In [25]:
user_query = "What is my out of pocket maximum and what is my PCP copay?"
index_query_1 = "what is my PCP copay?"
index_query_2 = "What is my out of pocket maximum and what is my PCP copay?"
index_query_3 = "What is my out of pocket maximum?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

user_query:What is my out of pocket maximum and what is my PCP copay?
index_query_1: what is my PCP copay?
index_query_2: What is my out of pocket maximum and what is my PCP copay?
index_query_3: What is my out of pocket maximum?
index_query_4: What types of home health care are covered?
index_query_5: Is there a limit on home health care visits?


SimilarityData(is_similar1=False, is_similar2=True, is_similar3=False, is_similar4=False, is_similar5=False, Reason1='The indexed question only addresses the copay for the PCP and does not mention the out of pocket maximum, making it a partial match.', Reason2='The indexed question matches both parts of the user query, addressing both the out of pocket maximum and the PCP copay.', Reason3='The indexed question only addresses the out of pocket maximum and does not mention the PCP copay, making it a partial match.', Reason4='The indexed question is unrelated to the user query, which focuses on out of pocket maximum and PCP copay.', Reason5='The indexed question is unrelated to the user query, which focuses on out of pocket maximum and PCP copay.')

In [46]:
def get_true_index(response):
    true_indices = [i for i, is_similar in enumerate([response.is_similar1, response.is_similar2, response.is_similar3, response.is_similar4, response.is_similar5], start=0) if is_similar]
    return true_indices

true_indices = get_true_index(convert_similarity_data_to_compare_questions(response))
print(true_indices)

AttributeError: 'compare_questions' object has no attribute 'is_similar1'

In [52]:
[i[1] for i in convert_similarity_data_to_compare_questions(response)]

[['NO', 'YES', 'NO', 'NO', 'NO']]

In [ ]:
similarity_results = [
    response.is_similar1,
    response.is_similar2,
    response.is_similar3,
    response.is_similar4,
    response.is_similar5
]

print(similarity_results)

In [38]:
# [index for index, result in enumerate(response) if result. ] 
for res in response:
    print(res[0])

is_similar1
is_similar2
is_similar3
is_similar4
is_similar5
Reason1
Reason2
Reason3
Reason4
Reason5


In [42]:
class SimilarityData(BaseModel):  
    is_similar1: Literal[True, False]  
    is_similar2: Literal[True, False]  
    is_similar3: Literal[True, False]  
    is_similar4: Literal[True, False]  
    is_similar5: Literal[True, False]  
    Reason1: str
    Reason2: str
    Reason3: str
    Reason4: str
    Reason5: str

from typing import List
class compare_questions(BaseModel):

    is_similar: List[Literal["YES", "NO", "YES", "NO", "YES"]]

In [43]:
def convert_similarity_data_to_compare_questions(similarity_data: SimilarityData) -> compare_questions:
    return compare_questions(
        is_similar=[
            "YES" if similarity_data.is_similar1 else "NO",
            "YES" if similarity_data.is_similar2 else "NO",
            "YES" if similarity_data.is_similar3 else "NO",
            "YES" if similarity_data.is_similar4 else "NO",
            "YES" if similarity_data.is_similar5 else "NO"
        ]
    )

NameError: name 'similarity_data' is not defined